In [86]:
import sqlite3
from collections import defaultdict

connection = sqlite3.connect('nba.sqlite')
cursor = connection.cursor()   

In [50]:
cursor.execute("SELECT * FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('table', 'Salaries', 'Salaries', 2, 'CREATE TABLE "Salaries" (\n\t"Player"\tTEXT,\n\t"Team"\tTEXT,\n\t"Salary"\tREAL\n)'), ('table', 'Stats', 'Stats', 3, 'CREATE TABLE "Stats" (\n\t"Year"\tREAL,\n\t"Player"\tTEXT,\n\t"Position"\tTEXT,\n\t"Age"\tREAL,\n\t"Team"\tTEXT,\n\t"Games"\tREAL,\n\t"GamesStarted"\tINTEGER,\n\t"MinutesPlayed"\tREAL,\n\t"3Points"\tINTEGER,\n\t"3PAttempts"\tINTEGER,\n\t"2Points"\tREAL,\n\t"2PAttempts"\tREAL,\n\t"FreeThrows"\tREAL,\n\t"FTAttempts"\tREAL,\n\t"Fouls"\tREAL,\n\t"Points"\tREAL\n)')]


In [4]:
def query(q):
    cursor.execute(q)
    return cursor.fetchall()

In [45]:
# Para cada equipo sumo los fouls de cada uno de sus participantes
# si estos perteneces a los 5 mas viejos del equipo
# finalmente orderno los equipos por la cantidad de fouls 
q = f"""
SELECT *
FROM(
    SELECT Team, SUM(Fouls) AS ss
    FROM Stats AS S
    WHERE Player IN (
        SELECT Player
        FROM Stats
        WHERE Team = S.Team
        ORDER BY Age DESC
        LIMIT 5
    )
    GROUP BY Team
    ORDER BY ss DESC
    LIMIT 5
)
ORDER BY ss ASC
"""

query(q)

[('TOR', 576.0),
 ('NYK', 588.0),
 ('POR', 661.0),
 ('BOS', 686.0),
 ('MEM', 698.0)]

In [97]:
q = f"""
SELECT * FROM Stats
"""

teams = defaultdict(list)
for item in query(q):
    (Year, Player, Position, Age, Team, Games, GamesStarted, MinutesPlayed, TPoints,
     TPAttempts, DPoints, DPAttempts, FreeThrows, FTAttempts, Fouls, Points) = item
    # obtengo 
    teams[Team].append((Player, float(Age), float(Fouls)))
    
tot = []
for t in teams:
    tot.append((t, sum([x[2] for x in sorted(teams[t], key=lambda x: x[1], reverse=True)[:5]])))

tot = sorted(tot, key=lambda x: x[1])[-5:]
tot    

[('TOR', 576.0),
 ('NYK', 588.0),
 ('POR', 661.0),
 ('BOS', 686.0),
 ('MEM', 698.0)]

In [116]:
# obtengo todos los participantes que han hecho mas de 1 3-puntos
players = f"""
SELECT Player
FROM Stats
GROUP BY Player
HAVING SUM("3Points") > 0
"""
# Para cada jugador,divido su salario por la suma de sus puntos,
# si el jugador pertenece a los jugadores encontrados anteriormente
# y los ordeno en cuanto a su salario
q = f"""
SELECT Sa.Player, Sa.Salary/SUM(St.Points) AS Ptot, Sa.Salary AS Tot
FROM Salaries AS Sa, Stats AS St
WHERE Sa.Player IN ({players}) AND St.Player=Sa.Player
GROUP BY Sa.Player
ORDER BY Tot DESC
LIMIT 10
"""

query(q)

[('Jonas Valanciunas', 16121.663190823774, 15460675.0),
 ('Taj Gibson', 8274.231678486998, 14000000.0),
 ('Michael Kidd-Gilchrist', 17496.63526244953, 13000000.0),
 ('Shaun Livingston', 19774.570694087404, 7692308.0),
 ('Lance Stephenson', 17131.147540983606, 4180000.0),
 ('Tarik Black', 8590.078328981723, 3290000.0),
 ('Quincy Acy', 3850.310810810811, 1709538.0),
 ('Willie Reed', 4217.192513368984, 1577230.0),
 ("DeAndre' Bembry", 15516.831683168317, 1567200.0),
 ("Johnny O'Bryant", 20056.644736842107, 1524305.0)]

In [123]:
q = f"""
SELECT * 
FROM Stats, Salaries
WHERE Stats.Player = Salaries.Player
"""

players = defaultdict(list)
for item in query(q):
    (Year, Player, Position, Age, Team, Games, GamesStarted, MinutesPlayed, TPoints,
     TPAttempts, DPoints, DPAttempts, FreeThrows, FTAttempts, Fouls, Points, _, _, Salary) = item
        
    players[Player].append((float(Salary), float(Points or 0), float(TPoints or 0)))
    
tot = []
for p, v in players.items():
#     print(p, v)
    if sum(x[2] for x in v) > 0:
        tot.append((p, v[0][0]/sum([x[1] for x in v]), v[0][0]))
    
tot = sorted(tot, key=lambda x: x[2], reverse=True)[:10]
tot

[('Jonas Valanciunas', 16121.663190823774, 15460675.0),
 ('Taj Gibson', 8274.231678486998, 14000000.0),
 ('Michael Kidd-Gilchrist', 17496.63526244953, 13000000.0),
 ('Shaun Livingston', 19774.570694087404, 7692308.0),
 ('Lance Stephenson', 17131.147540983606, 4180000.0),
 ('Tarik Black', 8590.078328981723, 3290000.0),
 ('Quincy Acy', 3850.310810810811, 1709538.0),
 ('Willie Reed', 4217.192513368984, 1577230.0),
 ("DeAndre' Bembry", 15516.831683168317, 1567200.0),
 ("Johnny O'Bryant", 20056.644736842107, 1524305.0)]